In [21]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sys

import folium

import src.mex_helper as mex_helper
import src.mex as mex
import src.mex.regions2010 as region
import src.mex.tower as tower
import src.tower_interpolation as tw_int
import src.utils.map_vis as mvis

from src.utils.map_vis import time_slider_choropleth, geojson_per_row
if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')


In [3]:
import pandas as pd
import geopandas as gp

# load tower stats

In [4]:
call_direction = 'out+in'
aver = mex_helper.stat_tw_dow_aver_hr_uniq_user(call_direction)
tw_avg_wd = pd.DataFrame(aver['wd']).T

loading cached tw average stats/stat_tw_dow_aver_hr_uniq_user-out+in.pickle


# interpolation

## to ageb

In [86]:
t2ageb_area = tw_int.to_mpa_agebs('area')
t2ageb_area.set_index('ageb', inplace=True)

to_map_agebs loading existing file data/mex_tw_intpl/tower_to_mpa_agebs_by_area.csv


In [87]:
ageb_area_avg = tw_int.interpolate_stats(tw_avg_wd, t2ageb_area)

In [201]:
t2ageb_pop = tw_int.to_mpa_agebs('pop')
t2ageb_pop.set_index('ageb', inplace=True)

to_map_agebs loading existing file data/mex_tw_intpl/tower_to_mpa_agebs_by_pop.csv


In [202]:
ageb_pop_avg = tw_int.interpolate_stats(tw_avg_wd, t2ageb_pop)

## to grid

In [285]:
t2g1000_area = tw_int.to_mpa_grids(1000, by='area', per_mun=False)
t2g1000_area.set_index('grid', inplace=True)
avg_g1000_area = tw_int.interpolate_stats(tw_avg_wd, t2g1000_area)

to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g1000_by_area.csv


In [286]:
t2g1000pop = tw_int.to_mpa_grids(1000, by='pop', per_mun=False)
t2g1000pop.set_index('grid', inplace=True)
avg_g1000pop = tw_int.interpolate_stats(tw_avg_wd, t2g1000pop)

to_map_grids loading existing file data/mex_tw_intpl/tower_to_mpa_g1000_by_pop.csv


# load geom

In [5]:
zms = region.mpa_all(to_4326=True)

In [6]:
# AGEBs

mapping = region.ageb_ids_per_mpa()
zms_agebs = region.agebs(mapping.mun_id.unique(), to_4326=True)

In [7]:
# Towers

tvor = tower.voronoi(to_4326=True)
tvor_x_zms = tower.voronoi_x_region('mpa')
tvor_x_zms = tvor_x_zms[tvor_x_zms.CVE_SUN.isin(zms.index.astype(str))]
zms_tvors= tvor.loc[set(tvor_x_zms.gtid)]

zms_tvors['sum_count']=tw_avg_wd.sum(axis=1)
zms_tvors.sum_count.fillna(0,inplace=True)

for cve_sun, gb in tvor_x_zms.groupby('CVE_SUN'):
    zms_tvors.loc[gb.gtid, 'color'] = zms_tvors.loc[gb.gtid].sum_count/zms_tvors.loc[gb.gtid].sum_count.max()

loading existing mexico tower voronoi file: data/mex_tw_intpl/voronoi.geojson


In [17]:
zms_g1000 = region.mpa_grids(1000,per_mun=False)

reading existing grids


In [18]:
zms_g1000_mun = region.mpa_grids(1000,per_mun=True)

reading existing grids
dividing grids by municipality
computing the intersection between p1 grid and p2 mgmu
computing area of the intersections
computing the weight


# hotspot 

In [88]:
from src.utils import loubar_thres
import numpy as np
def keep_hotspot(avg):
    hot_only = avg.copy()
    for h in avg:
        arr = avg[h]
        if hotspot_type=='loubar':
            _, arr_thres = loubar_thres(arr, is_sorted=False)
        elif hotspot_type=='average':
            arr_thres = np.mean(arr)
        else:
            raise ValueError('hotspot type', hotspot_type,'not implemented')
        hot_only[h][hot_only[h] <= arr_thres] = 0
        # print(h, loubar, arr_thres)
    return hot_only


In [89]:
hotspot_type='loubar'

## zms grids

In [290]:
by = 'area'
side=1000
avg_g = {
    (1000,'pop'): avg_g1000pop,
    (1000,'area'): avg_g1000_area
}[(side,by)]
grids = {
    1000: grids1000
}[side]


In [291]:
print(by,side)
zms_grids_per = []
for sun, sun_grids in grids.groupby('metropolitan'):
    sun_grids = sun_grids.copy()
    sun_grids_avg = avg_g.loc[sun_grids.index].copy()
    sun_grids_avg = sun_grids_avg.fillna(0)
    
    hot = keep_hotspot(sun_grids_avg)    
    sun_grids['CVE_SUN'] = sun
    sun_grids['persistency'] = (hot!=0).sum(axis=1)
    sun_grids['sum_count'] = sun_grids_avg.sum(axis=1)
    sun_grids = sun_grids[sun_grids.persistency!=0]
    zms_grids_per.append(sun_grids)

zms_grids_per = pd.concat(zms_grids_per)
zms_grids_per['color'] = zms_grids_per.persistency/24


area 1000


In [292]:

m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms.reset_index(), name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)

mvis.geojson_per_row_color_col(zms_grids_per.reset_index(), name='per', 
                               tip_cols=['id', 'metropolitan', 'persistency', 'sum_count'],
                              color_col='color',some_map=m)

mvis.geojson_per_row_color_col(zms_tvors.reset_index(), name='tower', tip_cols=['gtid', 'sum_count'],
                              color_col='color',some_map=m)
folium.LayerControl().add_to(m)
m.save(f'maps/mex_mpa_grid/{side}_{by}_persistency.html')

## zms agebs

In [203]:
zms_agebs_persistence = []

for cve_sun, group in mapping.groupby('CVE_SUN'):
#     lon,lat = zms.loc[cve_sun].geometry.centroid.coords[0]
    zm_ageb= zms_agebs.loc[group.ageb_id].copy()
    zm_ageb_avg = ageb_pop_avg.loc[group.ageb_id].copy()
    hot = keep_hotspot(zm_ageb_avg)    
    zm_ageb['CVE_SUN'] = cve_sun
    zm_ageb['persistency'] = (hot!=0).sum(axis=1)
    zm_ageb['sum_count'] = zm_ageb_avg.sum(axis=1)
    zm_ageb = zm_ageb[zm_ageb.persistency!=0]
    zms_agebs_persistence.append(zm_ageb)

zms_agebs_persistence = pd.concat(zms_agebs_persistence)

zms_agebs_persistence['color'] = zms_agebs_persistence.persistency/24

In [205]:

m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms.reset_index(), name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)

mvis.geojson_per_row_color_col(zms_agebs_persistence.reset_index(), name='per', 
                               tip_cols=['ageb_id', 'pobtot', 'CVE_SUN', 'persistency', 'sum_count'],
                              color_col='color',some_map=m)
mvis.geojson_per_row_color_col(zms_tvors.reset_index(), name='tower', tip_cols=['gtid', 'sum_count'],
                              color_col='color',some_map=m)
folium.LayerControl().add_to(m)
m.save('maps/mex_mpa_ageb_pop/persistency.html')

## groupby mun urban

In [293]:
mapping_urb = mapping[mapping['Type']=="Urban"]

In [294]:
mun_agebs_persistence = []

for mid, gb in mapping_urb.groupby('mun_id'):
#     lon,lat = zms.loc[cve_sun].geometry.centroid.coords[0]
    if len(gb)<10:
        print(mid, len(gb))
        continue
    mun_ageb= zms_agebs.loc[gb.ageb_id].copy()
    mun_ageb_avg = ageb_area_avg.loc[gb.ageb_id].copy()
    hot = keep_hotspot(mun_ageb_avg) 
    mun_ageb['CVE_SUN'] = gb.CVE_SUN.iloc[0]
    mun_ageb['persistency'] = (hot!=0).sum(axis=1)
    mun_ageb['sum_count'] = mun_ageb_avg.sum(axis=1)
    mun_ageb = mun_ageb[mun_ageb.persistency!=0]
    mun_agebs_persistence.append(mun_ageb)

mun_agebs_persistence = pd.concat(mun_agebs_persistence)

mun_agebs_persistence['color'] = mun_agebs_persistence.persistency/24

08004 9
13022 4
13039 9
13070 9
15006 4
15012 5
15017 8
15022 8
15027 6
15030 6
15034 2
15038 1
15043 6
15044 8
15046 3
15055 3
15068 8
15069 3
15072 6
15073 8
15075 6
15083 5
15089 7
15094 8
15098 3
15112 8
15115 6
15125 3
17009 7
17026 7
20083 5
20087 3
20115 7
20157 6
20174 1
20227 8
20293 9
20338 6
20350 8
20375 8
20403 2
20409 5
20519 3
20539 4
21060 3
21090 9
21122 4
21125 3
21136 2
21149 9
21163 6
21181 3
22008 3
29001 5
29002 7
29009 5
29017 5
29018 9
29022 1
29027 7
29029 4
29032 4
29035 3
29042 5
29048 9
29049 5
29050 4
29051 2
29053 4
29054 2
29056 5
29057 4
29058 4
29059 2
29060 3
30033 6
30059 6
30074 4
30082 7
30090 9
30093 3
30099 9
30120 6
30136 1
30182 5
30185 1
30196 4
30199 5
31013 6
31100 6


In [254]:
zms_mgm = region.municipalities(mun_ids, to_4326=True)

In [295]:

m = folium.Map(location=[mex.clat, mex.clon], zoom_start=5)
mvis.geojson_per_row(zms.reset_index(), name='metro', tip_cols=['NOM_SUN', 'CVE_SUN', 'pobtot'], color='grey', some_map=m)
mvis.geojson_per_row(zms_mgm.reset_index(), name='mu', tip_cols=['NOM_MUN', 'mun_id'], color='orange', some_map=m)

mvis.geojson_per_row_color_col(mun_agebs_persistence.reset_index(), name='per', 
                               tip_cols=['ageb_id', 'pobtot', 'CVE_SUN', 'persistency', 'sum_count'],
                              color_col='color',some_map=m)
mvis.geojson_per_row_color_col(zms_tvors.reset_index(), name='tower', tip_cols=['gtid', 'sum_count'],
                              color_col='color',some_map=m)
folium.LayerControl().add_to(m)
m.save('maps/mex_mpa_ageb_area/persistency_urb_per_municipality.html')